In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import datetime
import tqdm

# load csv
df = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\(2) prepare for the new dataset\intermediate\ACLED_GOSIF_GPP_CHIRPS_GOSIF_GPP_SD.csv')

# load food price
food_price = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\2.merge_data\intermediate\FAO_elevation_soil_sum.csv')

# covert date in df to datetime
df['date'] = pd.to_datetime(df['date'])
# convert date in food_price to datetime
food_price['date'] = pd.to_datetime(food_price['date'])
from shapely.geometry import Point

food_price['geometry'] = food_price.apply(lambda row: Point(row.long, row.lat), axis=1)
geo_df = gpd.GeoDataFrame(food_price, geometry='geometry')

# drop long and lat 
geo_df = geo_df.drop(['long', 'lat'], axis=1)

# rearrange the columns
geo_df = geo_df[['date', 'geometry', 'elevation', 'soil', 'price']]
# convert geo_df date to datetime
geo_df['date'] = pd.to_datetime(geo_df['date'])
from shapely import wkt

# Convert the 'geometry' column to shapely geometry
df['geometry'] = df['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# extract distinct date from gdf
date = gdf['date'].unique()


In [61]:
# create a empty dataframe
result = pd.DataFrame()

# loop through each date
for i in tqdm.tqdm(range(len(date))):
    gdf_1 = gdf[gdf['date'] == date[i]]
    geo_df_1 = geo_df[geo_df['date'] == date[i]]

    merged_gdf = gpd.sjoin(gdf_1, geo_df_1, how='left', op='intersects')

    # drop index_right date_right
    merged_gdf = merged_gdf.drop(['index_right', 'date_right'], axis=1)

    # rename date_left to date
    merged_gdf = merged_gdf.rename(columns={'date_left': 'date'})
    # convert geometry in merged_gdf to string
    merged_gdf['geometry'] = merged_gdf['geometry'].astype(str)

    # convert geometry in gdf_1 to string
    gdf_1['geometry'] = gdf_1['geometry'].astype(str)

    # extract distinct date and geometry from gdf_1
    gdf_1_d = gdf_1[['date', 'geometry']]

    # merge gdf_1 and merged_gdf on date and geometry
    merged_gdf = pd.merge(gdf_1_d, merged_gdf, on=['date', 'geometry'], how='left')

    # concatentate merged_gdf to result
    result = pd.concat([result, merged_gdf])




  0%|          | 0/72 [00:00<?, ?it/s]c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\geopandas\geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\geopandas\geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 

In [63]:
# save result to csv
result.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\ACLED_GOSIF_GPP_CHIRPS_GOSIF_GPP_SD_FAO.csv', index=False)